In [7]:
import numpy as np
import cv2

from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten,Conv2D
from tensorflow.keras.optimizers import Adam 
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [8]:
# Normalization: Scaling data to the range of 0-1 is traditionally referred to as normalization.
train_data_gen = ImageDataGenerator(
    rescale=1.0/255,
)

test_data_gen = ImageDataGenerator(
    rescale=1.0/255,
)


# Making train data from the set of images.
train_data = train_data_gen.flow_from_directory(
    directory='data-set/train',
    target_size= (48,48), #Change later to find the values.
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)
# Making test data from the set of images.
test_data = train_data_gen.flow_from_directory(
    directory='data-set/test',
    target_size= (48,48), #Change later to find the values.
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [15]:
# Model setup
model = Sequential()

model.add(Conv2D(
    filters=32,
    kernel_size=(4, 4),
    strides=(1, 1),  # (1,1) defines strides for width, and height
    input_shape=(48, 48, 1),
    padding='valid',
    activation='relu'
))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [10]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


In [17]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(
    learning_rate=0.00001, decay=1e-6), metrics=['accuracy'])
model.fit(train_data, steps_per_epoch=train_data.samples//128,
                    epochs=50, validation_data=test_data, validation_steps=test_data.samples//128)

Epoch 1/50
168/224 [=====================>........] - ETA: 29s - loss: 1.8271 - accuracy: 0.2478

In [ ]:
emotion_model.save_weights('model.h5')

In [ ]:
# define a video capture object
vid = cv2.VideoCapture(0)
while(True):

    # Capture the video frame
    # by frame
    ret, frame = vid.read()

    if not ret:
        break

    face_region = cv2.CascadeClassifier(
        "haarcascade_frontalface_default.xml"
    )
    # converting to grayscale, as IT take value in opposite direction BGR, using grayscale
    # make it easy to compute as it only has one channel only.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_region.detectMultiScale(
        gray, scaleFactor=1.3, minNeighbors=5)

    if faces == ():
        print("No faces found")

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (127, 0, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        cropped_img = np.expand_dims(np.expand_dims(
            cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        max_index = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[max_index], (x+15, y+20),
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (127, 0, 255), 2, cv2.LINE_4)
    # Display the resulting frameq
    cv2.imshow("frame", frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()


